In [71]:
import numpy as np
import pandas as pd 
import os
import sys
import cv2 

import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect
import random

In [72]:
!pwd

/home/modeling/UCF_CRIME/script


In [73]:
DATA_PATH = '/home/ubuntu/modeling/UCF_CRIME/data'
VIDEO_PATH = os.path.join(DATA_PATH, 'video')
ANNOTATION_PATH = os.path.join(DATA_PATH, 'annotation')

In [74]:
columns = ['video_name']

In [7]:
train_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Train_Annotation.txt'), header=None, names=columns)
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Test_Annotation.txt'), header=None, names=columns)

train_df

,video_name
0,Vandalism/Vandalism048_x264.mp4 7183
1,Training_Normal_Videos_Anomaly/Normal_Videos82...
2,Robbery/Robbery003_x264.mp4 984
3,Assault/Assault046_x264.mp4 952
4,Training_Normal_Videos_Anomaly/Normal_Videos57...
...,...
1605,Burglary/Burglary073_x264.mp4 2769
1606,Training_Normal_Videos_Anomaly/Normal_Videos05...
1607,Training_Normal_Videos_Anomaly/Normal_Videos24...
1608,Burglary/Burglary022_x264.mp4 2838


In [8]:
len(train_df), len(test_df)

(1610, 290)

In [9]:
anomal_list = ['Arson', 'Assault', 'Stealing', 'Training_Normal_Videos_Anomaly', 'Testing_Normal_Videos_Anomaly']

new_tr_video = []
new_te_video = []

for data in train_df['video_name']:
    ab_type, video_name = data.split('/')
    new_video_name, new_video_frame = video_name.split()[0], video_name.split()[1]
#     print(new_video_frame)
    if ab_type in anomal_list: 
        new_tr_video.append(ab_type + '/' + new_video_name + ' ' + new_video_frame)

In [10]:
for data in test_df['video_name']:
    ab_type, video_name = data.split('/')
    new_video_name, new_video_frame = video_name.split()[0], video_name.split()[1]
#     print(new_video_frame)
    if ab_type in anomal_list: 
        new_te_video.append(ab_type + '/' + new_video_name + ' ' + new_video_frame)

len(new_tr_video), len(new_te_video)

(983, 167)

In [11]:
new_train_df = pd.DataFrame(new_tr_video, columns=['video_name'])
new_test_df = pd.DataFrame(new_te_video, columns=['video_name'])

In [12]:
new_train_df

,video_name
0,Training_Normal_Videos_Anomaly/Normal_Videos82...
1,Assault/Assault046_x264.mp4 952
2,Training_Normal_Videos_Anomaly/Normal_Videos57...
3,Training_Normal_Videos_Anomaly/Normal_Videos81...
4,Training_Normal_Videos_Anomaly/Normal_Videos35...
...,...
978,Stealing/Stealing101_x264.mp4 2544
979,Training_Normal_Videos_Anomaly/Normal_Videos68...
980,Training_Normal_Videos_Anomaly/Normal_Videos05...
981,Training_Normal_Videos_Anomaly/Normal_Videos24...


In [13]:
new_train_df['video_name'][0]

'Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 1657'

In [14]:
new_test_df['video_name'][0]

'Arson/Arson022_x264.mp4 8640'

In [49]:
def return_idx(ab_type):
    if ab_type == 'Arson':
        return 0
    elif ab_type == 'Assault':
        return 1 
    elif ab_type == 'Stealing':
        return 2 
    elif ab_type == 'Training_Normal_Videos_Anomaly' or ab_type == 'Testing_Normal_Videos_Anomaly':
        return 3 

In [16]:
train_list = [] 
test_list = []

In [17]:
train_list

[]

In [18]:
new_train_df['video_name'][0]

'Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 1657'

In [19]:
for data in new_train_df['video_name']:
    video, total_frame = os.path.join(VIDEO_PATH, data.split()[0]), data.split()[1]
    # print(os.path.exists(video)) 
    frame_num = int(total_frame) - 32
    if os.path.exists(video):
        train_list.append(data.split()[0] + ' ' + str(frame_num))
    else:
        continue

In [20]:
train_list

['Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 1625',
 'Assault/Assault046_x264.mp4 920',
 'Training_Normal_Videos_Anomaly/Normal_Videos813_x264.mp4 2683',
 'Training_Normal_Videos_Anomaly/Normal_Videos354_x264.mp4 1763',
 'Training_Normal_Videos_Anomaly/Normal_Videos315_x264.mp4 1499',
 'Training_Normal_Videos_Anomaly/Normal_Videos803_x264.mp4 2694',
 'Training_Normal_Videos_Anomaly/Normal_Videos470_x264.mp4 1168',
 'Training_Normal_Videos_Anomaly/Normal_Videos253_x264.mp4 484',
 'Training_Normal_Videos_Anomaly/Normal_Videos082_x264.mp4 1166',
 'Training_Normal_Videos_Anomaly/Normal_Videos194_x264.mp4 4109',
 'Training_Normal_Videos_Anomaly/Normal_Videos124_x264.mp4 1362',
 'Training_Normal_Videos_Anomaly/Normal_Videos586_x264.mp4 1556',
 'Training_Normal_Videos_Anomaly/Normal_Videos225_x264.mp4 19909',
 'Training_Normal_Videos_Anomaly/Normal_Videos558_x264.mp4 1605',
 'Training_Normal_Videos_Anomaly/Normal_Videos468_x264.mp4 762',
 'Training_Normal_Videos_Anomaly/Normal_V

In [68]:
for data in new_test_df['video_name']:
    video, total_frame = os.path.join(VIDEO_PATH, data.split()[0]), data.split()[1]
    # print(os.path.exists(video))   
    # print(total_frame, str(int(total_frame) + 16))
    frame_num = int(total_frame) - 32
    if os.path.exists(video):
        test_list.append(data.split()[0] + ' ' + str(frame_num))
    else:
        continue

In [69]:
len(train_list), len(test_list)

(749, 167)

In [70]:
test_list

['Arson/Arson022_x264.mp4 8608',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_872_x264.mp4 498',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_873_x264.mp4 1767',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_248_x264.mp4 1108',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_478_x264.mp4 4470',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_891_x264.mp4 1768',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_828_x264.mp4 898',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_224_x264.mp4 6926',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_015_x264.mp4 448',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_210_x264.mp4 5376',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_247_x264.mp4 8179',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_881_x264.mp4 192',
 'Stealing/Stealing036_x264.mp4 2471',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_024_x264.mp4 1044',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_930_x264.mp4 3155',
 'Testing_Normal_Videos_Anomaly/Normal_Videos_940_x264.mp4 35985',
 'Testi

In [71]:
new_train_df = pd.DataFrame(train_list, columns=['video_name'])
new_test_df = pd.DataFrame(test_list, columns=['video_name'])

In [72]:
new_train_list = []
new_test_list = [] 

#### 평균 프레임 개수 구하기 

In [73]:
calc = 0 
for data in new_train_df['video_name']:
    # print(int(data.split()[1]))
    calc += int(data.split()[1])

calc // len(train_list)

9115

#### 총 프레임 개수가 12,000 이하인 영상에 대해서만 추출 

In [74]:
train_list = []
test_list = [] 

In [75]:
for data in new_train_df['video_name']:
    if int(data.split()[1]) <= 12000:
        train_list.append(data)
        
for data in new_test_df['video_name']:
    if int(data.split()[1]) <= 12000:
        test_list.append(data)

In [76]:
len(train_list), len(test_list)

(668, 160)

In [77]:
new_train_df = pd.DataFrame(train_list, columns=['video_name'])
new_test_df = pd.DataFrame(test_list, columns=['video_name'])

In [78]:
len(new_train_df), len(new_test_df)

(668, 160)

### 프레임 끊기 

In [79]:
new_train_list = []
new_test_list = []

In [80]:
def make_train_list(video_name, total_frame, ab_type):
    global new_train_list
    
    ab_num = return_idx(ab_type.split('/')[0])
    for idx in range(0, total_frame, 16):
        val = str(ab_type) + '/' + video_name + ' ' + str(idx) + ' ' + str(ab_num)
        new_train_list.append(val)

In [81]:
len(new_train_df), len(new_test_df)

(668, 160)

In [82]:
new_train_df['video_name'].head()

0    Training_Normal_Videos_Anomaly/Normal_Videos82...
1                      Assault/Assault046_x264.mp4 920
2    Training_Normal_Videos_Anomaly/Normal_Videos81...
3    Training_Normal_Videos_Anomaly/Normal_Videos35...
4    Training_Normal_Videos_Anomaly/Normal_Videos31...
Name: video_name, dtype: object

In [83]:
for idx, data in enumerate(new_train_df['video_name']):
    ab_type = data.split('/')[0]
    video_name, total_frame = data.split('/')[1].split(' ')[0], data.split('/')[1].split(' ')[1]
    make_train_list(video_name, int(total_frame), ab_type)

In [84]:
def make_test_list(video_name, total_frame, ab_type):
    global new_test_list
    
    ab_num = return_idx(ab_type.split('/')[0])
    for idx in range(0, total_frame, 16):
        val = str(ab_type) + '/' + video_name + ' ' + str(idx) + ' ' + str(ab_num)
        # print(idx)
        new_test_list.append(val)

In [85]:
for idx, data in enumerate(new_test_df['video_name']):
    ab_type = data.split('/')[0]
    video_name, total_frame = data.split('/')[1].split(' ')[0], data.split('/')[1].split(' ')[1]
    make_test_list(video_name, int(total_frame), ab_type)

In [86]:
len(new_train_list), len(new_test_list)

(119115, 25085)

In [87]:
train_df = pd.DataFrame(new_train_list, columns=columns, index=None)
test_df = pd.DataFrame(new_test_list, columns=columns, index=None)

In [88]:
len(train_df), len(test_df)

(119115, 25085)

In [89]:
train_df['video_name'][1]

'Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 16 3'

In [90]:
train_df['video_name'][0]

'Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 0 3'

In [91]:
train_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Train.lst'), header=0, index=None)
test_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Test.lst'), header=0, index=None)

### Test Dataset file 만들기 

In [75]:
ANNOTATION_PATH = '/home/modeling/UCF_CRIME/data/annotation' 
DATA_PATH = '/home/modeling/UCF_CRIME/data'
VIDEO_PATH = os.path.join(DATA_PATH, 'video')

In [76]:
!pwd

/home/modeling/UCF_CRIME/script


In [77]:
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Test_Annotation.txt'), header=None, names=columns)

In [78]:
test_df

,video_name
0,Arson/Arson022_x264.mp4 8640 Arson 3500 4000 -...
1,RoadAccidents/RoadAccidents021_x264.mp4 155 Ro...
2,Testing_Normal_Videos_Anomaly/Normal_Videos_87...
3,Testing_Normal_Videos_Anomaly/Normal_Videos_87...
4,Explosion/Explosion008_x264.mp4 1748 Explosion...
...,...
285,Testing_Normal_Videos_Anomaly/Normal_Videos_31...
286,Shooting/Shooting034_x264.mp4 1409 Shooting 96...
287,Testing_Normal_Videos_Anomaly/Normal_Videos_89...
288,Burglary/Burglary079_x264.mp4 14853 Burglary 7...


### 이상행동이 있는 동영상 리스트 만들기 

In [79]:
test_video = []

In [80]:
for data in test_df['video_name']:
    if data.split()[3] == '-1' and data.split()[4] == '-1':
        continue 
    test_video.append(data)    

In [81]:
len(test_video)

140

In [82]:
test_video

['Arson/Arson022_x264.mp4 8640 Arson 3500 4000 -1 -1 ',
 'RoadAccidents/RoadAccidents021_x264.mp4 155 RoadAccidents 30 90 -1 -1 ',
 'Explosion/Explosion008_x264.mp4 1748 Explosion 1005 1245 -1 -1 ',
 'Shooting/Shooting015_x264.mp4 1713 Shooting 855 1715 -1 -1 ',
 'Shooting/Shooting021_x264.mp4 1275 Shooting 480 630 -1 -1 ',
 'Explosion/Explosion016_x264.mp4 963 Explosion 180 450 -1 -1 ',
 'Shooting/Shooting007_x264.mp4 1430 Shooting 45 165 -1 -1 ',
 'Burglary/Burglary005_x264.mp4 7729 Burglary 4710 5040 -1 -1 ',
 'Explosion/Explosion021_x264.mp4 782 Explosion 135 270 -1 -1 ',
 'Shooting/Shooting008_x264.mp4 1625 Shooting 75 315 -1 -1 ',
 'Arrest/Arrest039_x264.mp4 15835 Arrest 7215 10335 -1 -1 ',
 'Stealing/Stealing036_x264.mp4 2503 Stealing 1260 1590 -1 -1 ',
 'Vandalism/Vandalism015_x264.mp4 2982 Vandalism 2010 2700 -1 -1 ',
 'Shoplifting/Shoplifting005_x264.mp4 1967 Shoplifting 720 930 -1 -1 ',
 'Burglary/Burglary033_x264.mp4 1259 Burglary 60 330 -1 -1 ',
 'Explosion/Explosion029_x2

In [83]:
our_test_video = []

In [84]:
video_name = data.split('/')[1].split()

video_name

['Explosion017_x264.mp4', '1643', 'Explosion', '990', '1440', '-1', '-1']

In [85]:
def return_idx(ab_type):
    if ab_type == 'Arson':
        return 0
    elif ab_type == 'Assault':
        return 1 
    elif ab_type == 'Stealing':
        return 2 
    elif ab_type == 'Training_Normal_Videos_Anomaly' or ab_type == 'Testing_Normal_Videos_Anomaly':
        return 3 

In [86]:
anomal_list = ['Arson', 'Assault', 'Stealing', 'Training_Normal_Videos_Anomaly', 'Testing_Normal_Videos_Anomaly']
flag = False

for data in test_video:
    ab_type, video_name = data.split('/')
    #print(video_name)
    start_frame, end_frame = video_name.split()[3], video_name.split()[4]
#     if video_name.split()[5] != '-1' and video_name.split()[6] != '-1':
#         flag = True 
#         start_frame_2, end_frame_2 = video_name.split()[5],  video_name.split()[6]
    
    label = return_idx(ab_type)

    if ab_type in anomal_list:
        our_test_video.append(ab_type + '/' + video_name.split()[0] + ' ' + str(start_frame) + ' ' + str(end_frame) + ' ' + str(label))

In [87]:
our_test_video

['Arson/Arson022_x264.mp4 3500 4000 0',
 'Stealing/Stealing036_x264.mp4 1260 1590 2',
 'Arson/Arson018_x264.mp4 270 600 0',
 'Arson/Arson011_x264.mp4 150 420 0',
 'Assault/Assault011_x264.mp4 375 960 1',
 'Arson/Arson035_x264.mp4 600 900 0',
 'Assault/Assault006_x264.mp4 1185 8096 1',
 'Assault/Assault010_x264.mp4 11330 11680 1',
 'Stealing/Stealing058_x264.mp4 570 3660 2',
 'Arson/Arson010_x264.mp4 885 1230 0',
 'Arson/Arson009_x264.mp4 220 315 0',
 'Stealing/Stealing079_x264.mp4 2550 3210 2',
 'Arson/Arson016_x264.mp4 1000 1796 0',
 'Arson/Arson007_x264.mp4 2250 5700 0',
 'Arson/Arson041_x264.mp4 2130 3615 0',
 'Stealing/Stealing019_x264.mp4 2730 2790 2',
 'Stealing/Stealing062_x264.mp4 360 1050 2']

In [147]:
test_video = pd.DataFrame(our_test_video, index=None)

In [149]:
test_video.to_csv(os.path.join(ANNOTATION_PATH, 'test_video.csv'), header=None, index=None)

In [88]:
!pwd

/home/modeling/UCF_CRIME/script


#### 폴더 만들기

In [89]:
for data in our_test_video:
    file = data.split()[0].split('.')[0]
    file_name = os.path.join(TEST_DATASET_PATH, file)
    print(file_name)
    try: 
        assert(os.path.exists(folder_name))
    except:
        os.makedirs(file_name)

/home/modeling/UCF_CRIME/data/data/TEST_DATA/Arson/Arson022_x264


FileExistsError: [Errno 17] File exists: '/home/modeling/UCF_CRIME/data/data/TEST_DATA/Arson/Arson022_x264'

#### 프레임 자르기 

In [90]:
TEST_DATASET_PATH

'/home/modeling/UCF_CRIME/data/data/TEST_DATA'

In [125]:
video = '/home/modeling/UCF_CRIME/data/video/Arson/Arson022_x264'

video.split('/')[7]

''

In [138]:
def video_to_image(video, start_frame, end_frame):
    # video: '/home/modeling/UCF_CRIME/data/video/Arson/Arson022_x264.mp4' 
    cap = cv2.VideoCapture(video)
    idx = 0 
    success, image = cap.read()
    count = 0 
    # print(video)
    while success: 
        success, image = cap.read()    
        if success and idx >= int(start_frame) and idx < int(end_frame):
            folder_name = video.split('/')[6], video.split('/')[7].split('.')[0]   # Arson/Arson022_x264
            cv2.imwrite(os.path.join(TEST_DATASET_PATH, folder_name, str(count) + '.png'), image)
            count += 1 
        if cv2.waitKey(10) == 27:
            break
        
        if idx >= int(end_frame): 
            break 
        
        idx += 1
    cap.release()    

In [139]:
our_test_video

['Arson/Arson022_x264.mp4 3500 4000 0',
 'Stealing/Stealing036_x264.mp4 1260 1590 2',
 'Arson/Arson018_x264.mp4 270 600 0',
 'Arson/Arson011_x264.mp4 150 420 0',
 'Assault/Assault011_x264.mp4 375 960 1',
 'Arson/Arson035_x264.mp4 600 900 0',
 'Assault/Assault006_x264.mp4 1185 8096 1',
 'Assault/Assault010_x264.mp4 11330 11680 1',
 'Stealing/Stealing058_x264.mp4 570 3660 2',
 'Arson/Arson010_x264.mp4 885 1230 0',
 'Arson/Arson009_x264.mp4 220 315 0',
 'Stealing/Stealing079_x264.mp4 2550 3210 2',
 'Arson/Arson016_x264.mp4 1000 1796 0',
 'Arson/Arson007_x264.mp4 2250 5700 0',
 'Arson/Arson041_x264.mp4 2130 3615 0',
 'Stealing/Stealing019_x264.mp4 2730 2790 2',
 'Stealing/Stealing062_x264.mp4 360 1050 2']

In [140]:
data.split()[0]

'Stealing/Stealing062_x264.mp4'

In [141]:
data.split()[1], data.split()[2]

('360', '1050')

In [142]:
our_test_video[0].split()[1], our_test_video[0].split()[2]

('3500', '4000')

In [143]:
!pwd

/home/modeling/UCF_CRIME/script


In [144]:
VIDEO_PATH

'/home/modeling/UCF_CRIME/data/video'

In [145]:
for data in our_test_video:
    video_name = os.path.join(VIDEO_PATH, data.split()[0])
    start_frame, end_frame = data.split()[1], data.split()[2]
    # print(video_name)
    video_to_image(video_name, start_frame, end_frame)  

### 각 폴더에 있는 이미지 파일 16프레임씩 읽어서 예측 